In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


import os
import random
import numpy as np

In [9]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [10]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [11]:
train_data = datasets.MNIST('data',train=True,download=True,transform=data_transforms)
test_data = datasets.MNIST('data',train=False,download=True,transform=data_transforms)

100.1%

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


113.5%

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw


100.4%

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw


180.4%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw
Processing...
Done!


In [12]:
n_train_examples = int(len(train_data)*0.9)
n_valid_examples = len(train_data) - n_train_examples

train_data,valid_data = torch.utils.data.random_split(train_data,[n_train_examples,n_valid_examples])

In [19]:
print("Number of training examples: {}".format(len(train_data)))
print("Number of validation examples: {}".format(len(valid_data)))
print("Number of training examples: {}".format(len(test_data)))

Number of training examples: 54000
Number of validation examples: 6000
Number of training examples: 10000


In [22]:
BATCH_SIZE = 64
train_iterator = torch.utils.data.DataLoader(train_data,shuffle=True,batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data,shuffle=True,batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data,shuffle=True,batch_size=BATCH_SIZE)


In [29]:
class MLP(nn.Module):
    def __init__(self,hidden_neurons):
        super().__init__()
        self.hidden_neurons = hidden_neurons
        self.input_fc = nn.Linear(28*28,hidden_neurons[0])
        self.fcs = nn.ModuleList([nn.Linear(hidden_neurons[i],hidden_neurons[i+1]) for i in range(len(hidden_neurons)-1)])
        self.output_fc = nn.Linear(hidden_neurons[-1],10)
    
    def forward(self,x):
        #flatten 
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fcs)
        for i in range(len(self.hidden_neurons)-1):
            x = F.relu(self.fcs[i](x))
        x = self.output_fc(x)
        return x
            

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
model = MLP([1000,500,250]).to(device)

In [31]:
model

MLP(
  (input_fc): Linear(in_features=784, out_features=1000, bias=True)
  (fcs): ModuleList(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=250, bias=True)
  )
  (output_fc): Linear(in_features=250, out_features=10, bias=True)
)

In [ ]:
class MLP1():
    def __init__(self,hidden_layers):
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Linear(28*28,hidden_layers[0])
        self.